<a href="https://colab.research.google.com/github/LimJiYeonn/LimJiYeonn.github.io/blob/master/%EC%97%B0%EC%8A%B5_5%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import pandas as pd
x_train = pd.read_csv('x_train.csv',encoding='CP949')
y_train = pd.read_csv('y_train.csv')
x_test = pd.read_csv('x_test.csv',encoding='CP949')

#print(x_train.info())
#print(x_train.describe().T)

#1
id = pd.DataFrame(x_test['cust_id'])
x_train.drop(columns='cust_id', inplace=True)
y_train.drop(columns='cust_id', inplace=True)
x_test.drop(columns='cust_id', inplace=True)

#3 결측값
#print(x_train.isnull().sum())
#print(x_test.isnull().sum())
x_train['환불금액'].fillna(0,inplace=True)
x_test['환불금액'].fillna(0,inplace=True)

#7 범주화
#print(x_train['주구매상품'].unique())
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])
x_test['주구매상품'] = encoder.transform(x_test['주구매상품'])
x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])
x_test['주구매지점'] = encoder.transform(x_test['주구매지점'])
#print(x_train['주구매지점'].head(3))
#print(x_train.info().T)


#8 파생변수
condition = x_train['환불금액'] > 0
x_train.loc[condition,'환불금액_new'] = 1
x_train.loc[~condition,'환불금액_new'] = 0
condition = x_test['환불금액'] > 0
x_test.loc[condition,'환불금액_new'] = 1
x_test.loc[~condition,'환불금액_new'] = 0

#5 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
sc = pd.DataFrame(scaler.fit_transform(x_train))
#print(x_train.describe().T)


#연관있어 보이는 변수 상관관계 확인 -> 0.6이상 칼럼 삭제
#print(x_train[['총구매액','최대구매액','환불금액']].corr())
x_train.drop(columns='최대구매액',inplace=True)
x_test.drop(columns='최대구매액',inplace=True)


#2
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train,test_size=0.2)

from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=1000, max_depth=5, eval_metric = 'error')
model.fit(X_TRAIN,Y_TRAIN)
y_test_predict = model.predict(X_TEST)
y_train_predict = model.predict(X_TRAIN)
print(pd.DataFrame(y_test_predict).head(3))

from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_TEST, y_test_predict))
print(roc_auc_score(Y_TRAIN, y_train_predict))

y_test_proba = pd.DataFrame(model.predict_proba(x_test))
print(pd.DataFrame(y_test_proba).head(3))

# 결과
result = pd.concat([id,y_test_proba[1]],axis=1).rename(columns = {1:'gender'})
f = result.to_csv('sample_data/1.csv',index = False)
print(f)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


   0
0  1
1  0
2  0
0.5935220935220935
0.9946807095933402
          0         1
0  0.095134  0.904866
1  0.994803  0.005197
2  0.953449  0.046551
None
